### Import and config


In [ ]:

import warnings
warnings.filterwarnings("ignore")

import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import statsmodels.api as sm, statsmodels.formula.api as smf
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import mstats


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PowerTransformer, PolynomialFeatures
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

sns.set(style='whitegrid')

OUT = Path('datasets')
OUT.mkdir(exist_ok=True)
print('Outputs will be saved to', OUT)

# User settings
FORCE_RESPONSE = None       # set column name to force response for Investigation A
WINSORIZE = True
WINSOR_PCT = 0.01
MIN_ROWS_SEASON = 20


Outputs will be saved to datasets


### Load dataset1 and dataset2

In [ ]:

d1 = OUT / 'dataset1.csv'
d2 = OUT / 'dataset2.csv'

df1 = pd.read_csv(d1)
df2 = pd.read_csv(d2)
# standardize column names
df1.columns = df1.columns.str.strip().str.lower().str.replace(' ', '_')
df2.columns = df2.columns.str.strip().str.lower().str.replace(' ', '_')
print('df1 shape:', df1.shape, 'df2 shape:', df2.shape)
df1.head(2)


df1 shape: (907, 12) df2 shape: (2123, 7)


,start_time,bat_landing_to_food,habit,rat_period_start,rat_period_end,seconds_after_rat_arrival,risk,reward,month,sunset_time,hours_after_sunset,season
0,30/12/2017 18:37,16.000000,rat,30/12/2017 18:35,30/12/2017 18:38,108,1,0,0,30/12/2017 16:45,1.870833,0
1,30/12/2017 19:51,0.074016,fast,30/12/2017 19:50,30/12/2017 19:55,17,0,1,0,30/12/2017 16:45,3.100833,0


### Cell 3 — Cleaning & basic feature engineering


In [ ]:
# Sadia 
nums = ['bat_landing_to_food','seconds_after_rat_arrival','hours_after_sunset','rat_minutes','rat_arrival_number','food_availability','bat_landing_number']
for c in nums:
    if c in df1.columns: df1[c] = pd.to_numeric(df1[c], errors='coerce')
    if c in df2.columns: df2[c] = pd.to_numeric(df2[c], errors='coerce')
 
# Datetime parsing if present
for col in ['start_time','rat_period_start','rat_period_end','sunset_time']:
    if col in df1.columns:
        df1[col] = pd.to_datetime(df1[col], errors='coerce', dayfirst=True)
if 'time' in df2.columns:
    df2['time'] = pd.to_datetime(df2['time'], errors='coerce', dayfirst=True)
 
# rat_present for df1
df1['rat_present'] = np.nan
if 'seconds_after_rat_arrival' in df1.columns:
    df1.loc[df1['seconds_after_rat_arrival'].notna(), 'rat_present'] = (df1.loc[df1['seconds_after_rat_arrival'].notna(), 'seconds_after_rat_arrival'] >= 0).astype(int)
if set(['start_time','rat_period_start','rat_period_end']).issubset(df1.columns):
    mask = df1['rat_present'].isna() & df1['start_time'].notna() & df1['rat_period_start'].notna() & df1['rat_period_end'].notna()
    df1.loc[mask, 'rat_present'] = ((df1.loc[mask,'start_time'] >= df1.loc[mask,'rat_period_start']) & (df1.loc[mask,'start_time'] <= df1.loc[mask,'rat_period_end'])).astype(int)
df1['rat_present'] = df1['rat_present'].fillna(0).astype(int)
 
# vigilance_time
if 'bat_landing_to_food' in df1.columns:
    df1['vigilance_time'] = df1['bat_landing_to_food'].astype(float)
 
# season_label mapping
if 'season' in df1.columns:
    df1['season'] = pd.to_numeric(df1['season'], errors='coerce')
    df1['season_label'] = df1['season'].map({0:'winter',1:'spring'})
else:
    df1['season_label'] = df1.get('season_label', pd.NA)
 
# rat_activity_index in df2
if set(['rat_arrival_number','rat_minutes']).issubset(df2.columns):
    df2['rat_activity_index'] = df2['rat_arrival_number'].fillna(0) * df2['rat_minutes'].fillna(0)
elif 'rat_arrival_number' in df2.columns:
    df2['rat_activity_index'] = df2['rat_arrival_number'].fillna(0)
elif 'rat_minutes' in df2.columns:
    df2['rat_activity_index'] = df2['rat_minutes'].fillna(0)
else:
    df2['rat_activity_index'] = 0
 
 
print("Missing values in dataset1 (non-zero only):")
print(df1.isna().sum()[df1.isna().sum()>0].sort_values(ascending=False))
 
print("\nMissing values in dataset2 (non-zero only):")
print(df2.isna().sum()[df2.isna().sum()>0].sort_values(ascending=False))
 
 
print('Cleaning and basic feature engineering done.')
df1.head(2)


Missing values in dataset1 (non-zero only):
habit    41
dtype: int64

Missing values in dataset2 (non-zero only):
Series([], dtype: int64)
Cleaning and basic feature engineering done.


,start_time,bat_landing_to_food,habit,rat_period_start,rat_period_end,seconds_after_rat_arrival,risk,reward,month,sunset_time,hours_after_sunset,season,rat_present,vigilance_time,season_label
0,2017-12-30 18:37:00,16.000000,rat,2017-12-30 18:35:00,2017-12-30 18:38:00,108,1,0,0,2017-12-30 16:45:00,1.870833,0,1,16.000000,winter
1,2017-12-30 19:51:00,0.074016,fast,2017-12-30 19:50:00,2017-12-30 19:55:00,17,0,1,0,2017-12-30 16:45:00,3.100833,0,1,0.074016,winter


### Select response variable and EDA for Investigation A

In [ ]:

if FORCE_RESPONSE and FORCE_RESPONSE in df1.columns:
    response = FORCE_RESPONSE
else:
    candidates = [c for c in df1.select_dtypes(include=[np.number]).columns if c not in ['month','season','rat_present','hours_after_sunset']]
    response = 'bat_landing_to_food' if 'bat_landing_to_food' in candidates else (candidates[0] if candidates else 'vigilance_time')
print('Investigation A response chosen:', response)

# Basic descriptives
print('Descriptive stats (response):\n', df1[response].describe())
print('\nBy rat_present:')
print(df1.groupby('rat_present')[response].agg(['count','mean','median','std']))

# Plots
plt.figure(figsize=(6,3))
sns.histplot(df1[response].dropna(), bins=40, kde=True)
plt.title(f'Distribution of {response}'); plt.tight_layout(); plt.savefig(OUT/f'{response}_dist.png'); plt.close()

plt.figure(figsize=(6,3))
sns.boxplot(data=df1, x='rat_present', y=response, showmeans=True)
plt.title(f'{response} by rat_present'); plt.tight_layout(); plt.savefig(OUT/f'{response}_by_rat_present.png'); plt.close()
print('Saved EDA plots.')


Investigation A response chosen: bat_landing_to_food
Descriptive stats (response):
 count    907.000000
mean      11.713134
std       27.644410
min        0.010238
25%        1.000000
50%        4.000000
75%       11.500000
max      443.000000
Name: bat_landing_to_food, dtype: float64

By rat_present:
             count       mean  median       std
rat_present                                    
1              907  11.713134     4.0  27.64441
Saved EDA plots.


### Utilities

In [ ]:

def safe_load(paths):
    for p in paths:
        if Path(p).exists():
            print(f"Loading {p}")
            return pd.read_csv(p)
    raise FileNotFoundError(f"None of the candidate paths exist: {paths}")

def winsorize_series(s, lower_q=0.01, upper_q=0.99):
    lo = s.quantile(lower_q)
    hi = s.quantile(upper_q)
    return s.clip(lo, hi)

def calculate_vif(df):
    X = sm.add_constant(df)
    vif = pd.DataFrame({
        "feature": df.columns,
        "VIF": [variance_inflation_factor(X.values, i + 1) for i in range(len(df.columns))]
    }).sort_values("VIF", ascending=False)
    return vif

def backward_elimination(X, y, p_thresh=0.05, verbose=True):
    X_sm = sm.add_constant(X)
    model = sm.OLS(y, X_sm).fit()
    while True:
        pvals = model.pvalues.drop('const')
        max_p = pvals.max()
        if max_p > p_thresh:
            drop_feat = pvals.idxmax()
            if verbose:
                print(f"Dropping {drop_feat} (p={max_p:.4f})")
            X = X.drop(columns=[drop_feat])
            X_sm = sm.add_constant(X)
            model = sm.OLS(y, X_sm).fit()
        else:
            break
    return X, model


CANDIDATE_PATHS = [
    "dataset1.csv", "Dataset1.csv",
    "datasets/dataset1.csv", 
]
OUT = Path("outputs")
OUT.mkdir(exist_ok=True)

df1 = safe_load(CANDIDATE_PATHS)

# Standardise column names
df1.columns = df1.columns.str.strip().str.lower()

Loading datasets/dataset1.csv


### Response variable for Investigation A

In [ ]:
response = "bat_landing_to_food"
if response not in df1.columns:
    raise KeyError(f"Response variable '{response}' not found in dataset columns: {df1.columns.tolist()}")

df = df1.copy()
df = df.dropna(subset=[response]).reset_index(drop=True)

In [ ]:
candidate_continuous = [
    "seconds_after_rat_arrival",
    "hours_after_sunset",
    "rat_minutes",
    "rat_arrival_number",
    "bat_landing_number",
    "food_availability"
]
# Add engineered continuous candidates if the raw columns exist
engineered_candidates = [
    "rat_presence_intensity",    # we'll compute if needed
    "foraging_efficiency"        # we'll compute if needed
]

# Build list of predictors that actually exist in df (and are numeric-capable)
predictors = []
for c in candidate_continuous:
    if c in df.columns:
        predictors.append(c)

# create engineered predictors if possible
if "seconds_after_rat_arrival" in df.columns:
    df["rat_presence_intensity"] = 1 / (1 + df["seconds_after_rat_arrival"].astype(float).replace(0, np.nan))
    df["rat_presence_intensity"] = df["rat_presence_intensity"].fillna(0)
    predictors.append("rat_presence_intensity")

# foraging_efficiency: reward / (landing_time + 1) if reward exists
if ("reward" in df.columns):
    df["foraging_efficiency"] = df["reward"].astype(float) / (df[response].astype(float) + 1)
    predictors.append("foraging_efficiency")

# Include binary/binary-like predictors that are meaningful
# e.g., risk, rat presence (if provided)
if "risk" in df.columns:
    # ensure numeric
    df["risk"] = pd.to_numeric(df["risk"], errors='coerce').fillna(0)
    predictors.append("risk")
if "rat_present" in df.columns:
    # convert to numeric
    df["rat_present"] = pd.to_numeric(df["rat_present"], errors='coerce').fillna(0)
    predictors.append("rat_present")
elif "seconds_after_rat_arrival" in df.columns:
    # derive rat_present from seconds_after_rat_arrival >= 0
    df["rat_present"] = (~df["seconds_after_rat_arrival"].isna()) & (df["seconds_after_rat_arrival"] >= 0)
    df["rat_present"] = df["rat_present"].astype(int)
    if "rat_present" not in predictors:
        predictors.append("rat_present")

# Remove duplicates and ensure predictors exist
predictors = [p for p in predictors if p in df.columns]
print("Predictor candidates used:", predictors)

# Drop any predictors that are constant or non-numeric after coercion
clean_preds = []
for p in predictors:
    df[p] = pd.to_numeric(df[p], errors="coerce")
    if df[p].nunique(dropna=True) > 1:
        clean_preds.append(p)
    else:
        print(f"Removing constant or invalid predictor: {p}")
predictors = clean_preds

# Final dataset for modeling
model_df = df[[response] + predictors].copy()
# Fill numeric NaNs by median (safer than mean)
for col in model_df.columns:
    if model_df[col].isna().any():
        model_df[col] = model_df[col].fillna(model_df[col].median())

Predictor candidates used: ['seconds_after_rat_arrival', 'hours_after_sunset', 'rat_presence_intensity', 'foraging_efficiency', 'risk', 'rat_present']
Removing constant or invalid predictor: rat_present


### Model Optimizer for Investigation A

In [ ]:
# -------------------------
# Winsorize outliers at 1% / 99%
# -------------------------
for col in model_df.select_dtypes(include=[np.number]).columns:
    model_df[col] = winsorize_series(model_df[col], 0.01, 0.99)

# -------------------------
# Log / Yeo-Johnson (Gaussianize) transform for skewed predictors
# -------------------------
# Use Yeo-Johnson (works with zero/negative)
pt = PowerTransformer(method="yeo-johnson", standardize=False)
X_raw = model_df[predictors].astype(float)
# detect skewness and transform only if skew > 0.75 (heuristic)
skews = X_raw.skew().abs()
to_transform = skews[skews > 0.75].index.tolist()
print("Features selected for Yeo-Johnson transform (skew>0.75):", to_transform)
if to_transform:
    X_trans = X_raw.copy()
    X_trans[to_transform] = pt.fit_transform(X_trans[to_transform])
else:
    X_trans = X_raw.copy()

# -------------------------
# Remove multicollinearity via iterative VIF removal (threshold 5.0)
# -------------------------
X_vif = X_trans.copy()
def iterative_vif_removal(X_df, thresh=5.0):
    X_work = X_df.copy()
    while True:
        vif = calculate_vif(X_work)
        max_vif = vif["VIF"].max()
        if max_vif > thresh:
            drop_feat = vif.sort_values("VIF", ascending=False)["feature"].iloc[0]
            print(f"Dropping {drop_feat} (VIF={max_vif:.2f})")
            X_work = X_work.drop(columns=[drop_feat])
        else:
            break
    return X_work

X_nmv = iterative_vif_removal(X_vif, thresh=5.0)
print("Predictors after VIF reduction:", list(X_nmv.columns))

# -------------------------
# Standardize predictors
# -------------------------
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_nmv), columns=X_nmv.columns, index=X_nmv.index)

# -------------------------
# Optionally add polynomial / interaction terms (only a few to avoid explosion)
# We'll add pairwise interactions for a small subset (top 4 features by variance)
# -------------------------
# choose up to 4 most variable features
vars_by_var = X_scaled.var().sort_values(ascending=False).index.tolist()
interaction_feats = vars_by_var[:4]
print("Creating interactions among:", interaction_feats)
for i in range(len(interaction_feats)):
    for j in range(i+1, len(interaction_feats)):
        a = interaction_feats[i]
        b = interaction_feats[j]
        name = f"{a}_x_{b}"
        X_scaled[name] = X_scaled[a] * X_scaled[b]

# -------------------------
# Backward feature selection by p-value on OLS (start with current X_scaled)
# -------------------------
X_for_model = X_scaled.copy()
y_final = model_df[response].astype(float)

X_selected, ols_full = backward_elimination(X_for_model, y_final, p_thresh=0.05, verbose=True)
print("Final features after backward elimination:", list(X_selected.columns))

Features selected for Yeo-Johnson transform (skew>0.75): ['seconds_after_rat_arrival', 'rat_presence_intensity', 'foraging_efficiency']
Dropping rat_presence_intensity (VIF=5.97)
Predictors after VIF reduction: ['seconds_after_rat_arrival', 'hours_after_sunset', 'foraging_efficiency', 'risk']
Creating interactions among: ['hours_after_sunset', 'foraging_efficiency', 'seconds_after_rat_arrival', 'risk']
Dropping foraging_efficiency_x_seconds_after_rat_arrival (p=0.9064)
Dropping risk (p=0.5934)
Dropping hours_after_sunset (p=0.4216)
Dropping seconds_after_rat_arrival (p=0.3859)
Dropping hours_after_sunset_x_foraging_efficiency (p=0.3344)
Dropping hours_after_sunset_x_risk (p=0.5334)
Dropping hours_after_sunset_x_seconds_after_rat_arrival (p=0.2702)
Dropping seconds_after_rat_arrival_x_risk (p=0.1156)
Final features after backward elimination: ['foraging_efficiency', 'foraging_efficiency_x_risk']


### Train and Split

In [ ]:
# Sadia 
X_tr, X_te, y_tr, y_te = train_test_split(X_selected, y_final, test_size=0.2, random_state=42)
 
# -------------------------
# Fit OLS (final) and evaluate
# -------------------------
X_tr_sm = sm.add_constant(X_tr)
final_ols = sm.OLS(y_tr, X_tr_sm).fit()
print("\nFinal OLS summary:")
print(final_ols.summary())
 
X_te_sm = sm.add_constant(X_te)
y_pred_ols = final_ols.predict(X_te_sm)
 
r2_ols = r2_score(y_te, y_pred_ols)
rmse_ols = np.sqrt(mean_squared_error(y_te, y_pred_ols))
mae_ols = mean_absolute_error(y_te, y_pred_ols)
print(f"\nFinal OLS Test metrics: R²={r2_ols:.4f}, RMSE={rmse_ols:.4f}, MAE={mae_ols:.4f}")
 
# -------------------------
# Regularized models (RidgeCV, LassoCV) for comparison & stability
# -------------------------
ridge = RidgeCV(alphas=np.logspace(-4, 4, 50), cv=5).fit(X_tr, y_tr)
y_pred_ridge = ridge.predict(X_te)
r2_ridge = r2_score(y_te, y_pred_ridge); rmse_ridge = np.sqrt(mean_squared_error(y_te, y_pred_ridge))
 
lasso = LassoCV(alphas=np.logspace(-4, 2, 50), cv=5, max_iter=10000).fit(X_tr, y_tr)
y_pred_lasso = lasso.predict(X_te)
r2_lasso = r2_score(y_te, y_pred_lasso); rmse_lasso = np.sqrt(mean_squared_error(y_te, y_pred_lasso))
 
print(f"Ridge Test metrics: R²={r2_ridge:.4f}, RMSE={rmse_ridge:.4f}, alpha={ridge.alpha_}")
print(f"Lasso Test metrics: R²={r2_lasso:.4f}, RMSE={rmse_lasso:.4f}, alpha={lasso.alpha_}")
 
# -------------------------
# Cross-validation for final OLS (10-fold)
# -------------------------
cv_scores = cross_val_score(LinearRegression(), X_selected, y_final, cv=10, scoring="r2")
print(f"10-fold CV R²: mean={cv_scores.mean():.4f}, std={cv_scores.std():.4f}")
 
# -------------------------
# Diagnostic plots: residuals vs fitted and QQ and Actual vs Predicted
# -------------------------
OUT.mkdir(exist_ok=True)
 
# OLS diagnostics on test set
residuals = y_te - y_pred_ols
fitted = y_pred_ols
 
plt.figure(figsize=(7,4))
sns.scatterplot(x=fitted, y=residuals, alpha=0.6)
plt.axhline(0, color="red", linestyle="--")
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Final OLS)")
plt.tight_layout()
plt.savefig(OUT / "A_resid_vs_fitted_two.png")
plt.close()
 
# QQ plot
plt.figure(figsize=(6,6))
sm.qqplot(residuals, line="45", fit=True)
plt.title("QQ plot (Final OLS residuals)")
plt.tight_layout()
plt.savefig(OUT / "A_qq_two_final.png")
plt.close()
 
# Actual vs predicted
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_te, y=y_pred_ols, alpha=0.6)
plt.plot([y_te.min(), y_te.max()], [y_te.min(), y_te.max()], color="red", linestyle="--")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs Predicted (Final OLS)")
plt.tight_layout()
plt.savefig(OUT / "A_actual_vs_predicted_final.png")
plt.close()
 
# -------------------------
# Save outputs & summary
# -------------------------
model_summary = final_ols.summary().as_text()
with open(OUT / "InvestigationA_final_OLS_summary.txt", "w") as f:
    f.write(model_summary)
 
results = {
    "final_ols_test_r2": float(r2_ols),
    "final_ols_rmse": float(rmse_ols),
    "ridge_test_r2": float(r2_ridge),
    "ridge_rmse": float(rmse_ridge),
    "lasso_test_r2": float(r2_lasso),
    "lasso_rmse": float(rmse_lasso),
    "cv_r2_mean": float(cv_scores.mean()),
    "cv_r2_std": float(cv_scores.std()),
    "num_obs": int(model_df.shape[0]),
    "final_predictors": list(X_selected.columns)
}
 
results
 
pd.DataFrame([results]).to_csv(OUT / "InvestigationA_results_summary.csv", index=False)
model_df.to_csv(OUT / "InvestigationA_model_data.csv", index=False)
 
print("\nOutputs saved to folder:", OUT.resolve())
print("Final model performance summary:")
print(pd.DataFrame([results]).T)


Final OLS summary:
                             OLS Regression Results                            
Dep. Variable:     bat_landing_to_food   R-squared:                       0.099
Model:                             OLS   Adj. R-squared:                  0.097
Method:                  Least Squares   F-statistic:                     39.74
Date:                 Thu, 16 Oct 2025   Prob (F-statistic):           4.24e-17
Time:                         01:04:30   Log-Likelihood:                -3212.1
No. Observations:                  725   AIC:                             6430.
Df Residuals:                      722   BIC:                             6444.
Df Model:                            2                                         
Covariance Type:             nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------

<Figure size 600x600 with 0 Axes>

# Investigation B Seasonal Effects

In [ ]:

# Cell 6 — Merge df1 and df2 for Investigation B
merged = df1.copy()
if set(['hours_after_sunset','month']).issubset(df1.columns) and set(['hours_after_sunset','month']).issubset(df2.columns):
    m1 = df1.sort_values('hours_after_sunset').reset_index(drop=True)
    m2 = df2.sort_values('hours_after_sunset').reset_index(drop=True)
    try:
        merged = pd.merge_asof(m1, m2, on='hours_after_sunset', by='month', direction='nearest', tolerance=0.5, suffixes=('','_sess'))
        print('Merged shape:', merged.shape)
    except Exception as e:
        print('merge_asof failed:', e)
        merged = df1.copy()
else:
    print('Merge not possible; using df1 copy.')

merged.to_csv(OUT/'merged_df_for_investigationB.csv', index=False)
merged.head(2)


Merged shape: (907, 18)


,start_time,bat_landing_to_food,habit,rat_period_start,rat_period_end,seconds_after_rat_arrival,risk,reward,month,sunset_time,hours_after_sunset,season,time,bat_landing_number,food_availability,rat_minutes,rat_arrival_number,rat_activity_index
0,17/05/2018 19:16,0.068724,NaN,17/05/2018 19:14,17/05/2018 19:16,121,0,0,5,17/05/2018 19:32,-0.261667,1,2018-05-03 18:52:00,31,3.896118,23.216667,1,23.216667
1,1/05/2018 19:09,4.000000,pick,1/05/2018 18:58,1/05/2018 19:17,618,0,1,5,1/05/2018 19:21,-0.198611,1,2018-05-07 19:25:00,32,3.350877,0.000000,0,0.000000


### Prepare merged dataset: winsorize, log-transform, interactions, scaling

In [ ]:

data = merged.copy()
response = 'bat_landing_to_food' if 'bat_landing_to_food' in data.columns else 'vigilance_time'
data = data[data[response].notna()].copy()
data = data[data[response] >= 0]

if WINSORIZE:
    data[response + '_winsor'] = mstats.winsorize(data[response].fillna(0), limits=(WINSOR_PCT, WINSOR_PCT))
else:
    data[response + '_winsor'] = data[response].fillna(0)

data['y_log'] = np.log1p(data[response + '_winsor'])

cont_candidates = ['rat_activity_index','rat_minutes','rat_arrival_number','bat_landing_number','food_availability','hours_after_sunset','seconds_after_rat_arrival']
cat_candidates = ['rat_present','risk','reward','habit']

predictors = [c for c in cont_candidates + cat_candidates if c in data.columns]

if 'rat_activity_index' in data.columns and 'rat_present' in data.columns:
    data['ratact_x_ratpresent'] = data['rat_activity_index'] * data['rat_present']
    predictors.append('ratact_x_ratpresent')

if 'food_availability' in data.columns and 'season' in data.columns:
    data['season_num'] = data['season'].map({0:0,1:1}) if 'season' in data.columns else data['season_label'].map({'winter':0,'spring':1})
    data['food_x_season'] = data['food_availability'] * data['season_num']
    predictors.append('food_x_season')

numeric_preds = [p for p in predictors if p in cont_candidates + ['ratact_x_ratpresent','food_x_season','seconds_after_rat_arrival']]
scaler = StandardScaler()
if numeric_preds:
    data[numeric_preds] = scaler.fit_transform(data[numeric_preds].fillna(0))

data.to_csv(OUT/'merged_scaled_for_modeling.csv', index=False)
print('Prepared merged dataset. predictors:', predictors)


Prepared merged dataset. predictors: ['rat_activity_index', 'rat_minutes', 'rat_arrival_number', 'bat_landing_number', 'food_availability', 'hours_after_sunset', 'seconds_after_rat_arrival', 'risk', 'reward', 'habit', 'food_x_season']


### Backward selection by AIC on y_log


In [ ]:

def backward_aic(df, response_col, predictors_list, categorical_prefix=None):
    best_preds = predictors_list.copy()
    categorical_prefix = categorical_prefix or []
    def formula(preds):
        terms = []
        for p in preds:
            if p in categorical_prefix:
                terms.append(f"C({p})")
            else:
                terms.append(p)
        return response_col + ' ~ ' + ' + '.join(terms) if terms else response_col + ' ~ 1'
    current_model = smf.ols(formula(best_preds), data=df).fit()
    current_aic = current_model.aic
    improved = True
    while improved and len(best_preds) > 0:
        improved = False
        candidates = []
        for p in best_preds:
            trial = [x for x in best_preds if x != p]
            try:
                m = smf.ols(formula(trial), data=df).fit()
                candidates.append((p, m.aic))
            except:
                candidates.append((p, np.inf))
        p_remove, best_aic = min(candidates, key=lambda x: x[1])
        if best_aic + 1e-6 < current_aic:
            best_preds.remove(p_remove)
            current_aic = best_aic
            improved = True
    return smf.ols(formula(best_preds), data=df).fit(), best_preds

categorical_vars = [c for c in ['rat_present','risk','reward','habit'] if c in data.columns]
initial_preds = [p for p in predictors if p in data.columns]
print('Initial preds:', initial_preds)
model_aic, selected_preds = backward_aic(data.dropna(subset=['y_log']), 'y_log', initial_preds, categorical_prefix=categorical_vars)
print('Selected preds:', selected_preds)
with open(OUT/'investigationB_selected_model_summary.txt','w') as f: f.write(model_aic.summary().as_text())


Initial preds: ['rat_activity_index', 'rat_minutes', 'rat_arrival_number', 'bat_landing_number', 'food_availability', 'hours_after_sunset', 'seconds_after_rat_arrival', 'risk', 'reward', 'habit', 'food_x_season']
Selected preds: ['rat_activity_index', 'rat_minutes', 'rat_arrival_number', 'food_availability', 'hours_after_sunset', 'risk', 'reward', 'habit', 'food_x_season']


### Final model diagnostics & performance 

In [ ]:

final_model = model_aic
print(final_model.summary())

resid = final_model.resid; fitted = final_model.fittedvalues
plt.figure(figsize=(6,3)); plt.scatter(fitted, resid, alpha=0.5); plt.axhline(0,color='r',linestyle='--'); plt.title('Final model resid vs fitted'); plt.tight_layout(); plt.savefig(OUT/'final_resid_vs_fitted.png'); plt.close()
sm.qqplot(resid, line='45'); plt.title('Final model QQ'); plt.tight_layout(); plt.savefig(OUT/'final_qq.png'); plt.close()

# VIF
try:
    exog = final_model.model.exog
    vif = pd.DataFrame({'variable': final_model.model.exog_names, 'VIF':[variance_inflation_factor(exog, i) for i in range(exog.shape[1])]})
    vif.to_csv(OUT/'final_vif.csv', index=False)
except Exception as e:
    print('VIF failed:', e)

# Performance metrics (safe)
y_true_log = data.loc[final_model.model.data.row_labels, 'y_log']
y_pred_log = final_model.fittedvalues
mask = np.isfinite(y_true_log) & np.isfinite(y_pred_log)
if mask.sum() > 0:
    rmse_log = np.sqrt(mean_squared_error(y_true_log[mask], y_pred_log[mask]))
    r2_log = final_model.rsquared
    rmse_orig = np.sqrt(mean_squared_error(np.expm1(y_true_log[mask]), np.expm1(y_pred_log[mask])))
    mae_orig = mean_absolute_error(np.expm1(y_true_log[mask]), np.expm1(y_pred_log[mask]))
else:
    rmse_log = r2_log = rmse_orig = mae_orig = np.nan

pd.DataFrame({'metric':['rmse_log','r2_log','rmse_orig','mae_orig'],'value':[rmse_log,r2_log,rmse_orig,mae_orig]}).to_csv(OUT/'final_model_performance.csv', index=False)
pd.DataFrame({'coef': final_model.params}).to_csv(OUT/'final_model_coeffs.csv')
print('Final model artifacts saved.')


                            OLS Regression Results                            
Dep. Variable:                  y_log   R-squared:                       0.448
Model:                            OLS   Adj. R-squared:                  0.387
Method:                 Least Squares   F-statistic:                     7.349
Date:                Thu, 16 Oct 2025   Prob (F-statistic):           3.52e-57
Time:                        01:04:31   Log-Likelihood:                -1105.1
No. Observations:                 866   AIC:                             2384.
Df Residuals:                     779   BIC:                             2799.
Df Model:                          86                                         
Covariance Type:            nonrobust                                         
                                                                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

### Seasonal models and performance comparison

In [ ]:

data['season_label'] = data['season_label'].astype(str)
winter_df = data[data['season_label']=='winter'].copy()
spring_df = data[data['season_label']=='spring'].copy()
print('Winter n=', winter_df.shape[0], 'Spring n=', spring_df.shape[0])

season_models = {}
for name, df_s in [('winter', winter_df), ('spring', spring_df)]:
    if df_s.shape[0] >= MIN_ROWS_SEASON:
        terms = [f'C({p})' if p in categorical_vars else p for p in selected_preds]
        formula_s = 'y_log ~ ' + ' + '.join(terms) if terms else 'y_log ~ 1'
        try:
            m = smf.ols(formula_s, data=df_s).fit()
            season_models[name] = m
            with open(OUT/f'investigationB_ols_{name}_summary.txt','w') as f: f.write(m.summary().as_text())
            print(name, 'model R2_log=', m.rsquared)
        except Exception as e:
            print('Fit failed for', name, e)
    else:
        print('Not enough rows for', name)

# Performance comparison (safe NaN handling)
perf_rows = []
for name, df_s in [('full', data), ('winter', winter_df), ('spring', spring_df)]:
    if name == 'full':
        m = final_model
        df_use = data.dropna(subset=['y_log'])
    else:
        m = season_models.get(name)
        df_use = df_s.dropna(subset=['y_log'])
    if m is None or df_use.empty:
        perf_rows.append({'Model':name,'R2_log':np.nan,'RMSE_log':np.nan,'RMSE_orig':np.nan,'NumObs':int(df_use.shape[0])}); continue
    y_true_log = df_use['y_log']
    y_pred_log = m.predict(df_use)
    mask = np.isfinite(y_true_log) & np.isfinite(y_pred_log)
    if mask.sum() == 0:
        perf_rows.append({'Model':name,'R2_log':np.nan,'RMSE_log':np.nan,'RMSE_orig':np.nan,'NumObs':int(df_use.shape[0])}); continue
    rmse_log = np.sqrt(mean_squared_error(y_true_log[mask], y_pred_log[mask]))
    rmse_orig = np.sqrt(mean_squared_error(np.expm1(y_true_log[mask]), np.expm1(y_pred_log[mask])))
    perf_rows.append({'Model':name,'R2_log':float(m.rsquared),'RMSE_log':float(rmse_log),'RMSE_orig':float(rmse_orig),'NumObs':int(df_use.shape[0])})

perf_df = pd.DataFrame(perf_rows)
perf_df.to_csv(OUT/'seasonal_model_performance_comparison.csv', index=False)
print(perf_df)

# Plot comparison
plt.figure(figsize=(7,4))
perf_df.set_index('Model')[['R2_log','RMSE_log']].plot(kind='bar', secondary_y='RMSE_log', rot=0)
plt.title('Seasonal Model Performance (R2_log and RMSE_log)'); plt.tight_layout(); plt.savefig(OUT/'seasonal_model_performance_plot.png'); plt.close()
print('Seasonal performance plot saved.')


KeyError: 'season_label'

In [ ]:
winter_model = season_models.get('winter')
spring_model = season_models.get('spring')

if winter_model is not None and spring_model is not None:
    winter_coefs = winter_model.params.rename('Winter Coef')
    spring_coefs = spring_model.params.rename('Spring Coef')

    coef_comparison = pd.concat([winter_coefs, spring_coefs], axis=1)
    coef_comparison['Difference (Spring - Winter)'] = coef_comparison['Spring Coef'] - coef_comparison['Winter Coef']

    coef_comparison.to_csv(OUT / 'seasonal_coef_comparison.csv')
    print('\nSeasonal Coefficient Comparison Table:')
    print(coef_comparison.round(3))
else:
    print("❌ Cannot compare coefficients: one or both seasonal models are missing.")


Seasonal Coefficient Comparison Table:
                              Winter Coef  Spring Coef  \
Intercept                           0.538        0.117   
C(risk)[T.1]                       -0.619        2.079   
C(reward)[T.1]                      0.366        0.776   
C(habit)[T.attack_rat]             -1.460          NaN   
C(habit)[T.bat]                     0.374       -0.200   
...                                   ...          ...   
C(habit)[T.rat_bat_fight]             NaN        1.599   
C(habit)[T.rat_disappear]             NaN       -0.375   
C(habit)[T.rat_pick]                  NaN       -0.107   
C(habit)[T.rat_pick_and_bat]          NaN       -2.730   
C(habit)[T.rat_to_bat]                NaN       -2.134   

                              Difference (Spring - Winter)  
Intercept                                           -0.421  
C(risk)[T.1]                                         2.698  
C(reward)[T.1]                                       0.410  
C(habit)[T.attack_r